In [1]:
# Import essential libraries and packages

import pandas as pd
import numpy as np
import math
import scipy as sp
import itertools
from pandas.tseries.offsets import DateOffset
import os  #provides functions for interacting with the operating system


from matplotlib import pyplot as plt
from IPython.display import Image
import altair as alt

import panel as pn
import datetime as dt
from altair import datum

alt.renderers.enable('default')
pn.extension('vega')

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# PLOT 2 

### 1 - Loading All Datasets

In [2]:
# Load counties datasets

df_alm = pd.read_pickle('df_alm.pickle')
df_con = pd.read_pickle('df_con.pickle')
df_mar = pd.read_pickle('df_mar.pickle')
df_nap = pd.read_pickle('df_nap.pickle')
df_sf  = pd.read_pickle('df_sf.pickle')
df_smt = pd.read_pickle('df_smt.pickle')
df_scl = pd.read_pickle('df_scl.pickle')
df_sol = pd.read_pickle('df_sol.pickle')
df_son = pd.read_pickle('df_son.pickle')

In [3]:
df_alm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 50
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Time              159 non-null    datetime64[ns]
 1   Alameda           147 non-null    float64       
 2   Prediction ARIMA  51 non-null     float64       
 3   lower_bond        51 non-null     float64       
 4   higher_bond       51 non-null     float64       
 5   Time2             159 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 8.7+ KB


### 2 - create the dataframe for 'Dec 2018'

In [4]:
dec2008 = [{'CA':283060 ,'Alameda':368240, 'Contra Costa':253790,'Marin':704545,'Napa':393477,'San Francisco':711363,
            'San Mateo':600000,'Santa Clara':512450,'Solano':217941,'Sonoma':338983,'Bay Area':359910}]


Dec_2008_df = pd.DataFrame(dec2008)

### 3 - Initial manipulation 

In [5]:
def df_manipulater_func (df):
    
    '''
        This function gets the DataFrame for each county and do these steps :
        - Add a Column as 'County' with the name of county in all rows.
        - Change the Price column name from county name to Actual
        - Change the Time2 Column to Date
        
    '''
    
    # Add a column as 'County' with the name of a county in all rows
    
    county_name = df.columns[1]
    df['County'] = county_name
    
    # change the price column from name of the county to 'Actual'
    
    df.rename(columns = {county_name:'Actual'}, inplace = True)
    
    # Change the 'Time2' column name to 'Date':
    df.rename(columns = {'Time2':'Date'}, inplace = True)

In [6]:
# create a list of all dfs of all counties 

df_list = [df_alm, df_con, df_mar, df_nap, df_sf, df_smt, df_scl, df_sol, df_son]

In [7]:
# apply manipulater function on all dfs:

for df_ in df_list: 
    df_manipulater_func(df_)

### 4 - Add year, quarter and month columns 

In [8]:
def year_month_quarter_add (df):
    
    df['Year'] = np.NaN
    df['Month'] = np.NaN
    df['Quarter'] = np.NaN
    df['Month Name'] = np.NaN
    
    for i in range(len(df)):
        
        df['Year'].iloc[i] = df['Time'].iloc[i].year
        df['Month'].iloc[i] = df['Time'].iloc[i].month
        df['Quarter'].iloc[i] = df['Time'].iloc[i].quarter


        
    df['Year'] = df['Year'].astype(int)
    df['Month'] = df['Month'].astype(int)
    df['Quarter'] = df['Quarter'].astype(int)
    
    # create month name column for charts
    df['Month Name'] = pd.to_datetime(df['Month'], format='%m').dt.month_name()

In [9]:
# apply year,month,quarter added function on all dfs:

for df_ in df_list: 
    year_month_quarter_add(df_)

In [10]:
df_alm.tail()

,Time,Actual,Prediction ARIMA,lower_bond,higher_bond,Date,County,Year,Month,Quarter,Month Name
46,2021-11-01,NaN,1238123.0,1165630.0,1316533.0,Nov 01 2021,Alameda,2021,11,4,November
47,2021-12-01,NaN,1223919.0,1152258.0,1301429.0,Dec 01 2021,Alameda,2021,12,4,December
48,2022-01-01,NaN,1216021.0,1144822.0,1293031.0,Jan 01 2022,Alameda,2022,1,1,January
49,2022-02-01,NaN,1257895.0,1184245.0,1337557.0,Feb 01 2022,Alameda,2022,2,1,February
50,2022-03-01,NaN,1325731.0,1248109.0,1409689.0,Mar 01 2022,Alameda,2022,3,1,March


### 5 - Add 'price Index', 'avg price index' ,'Growth Rate, 'avg Growth Rate' column (January of each year is 100)

In [11]:
def price_index_growth_rate(df, price_col='Actual'):

    ''' This function gets the DataFrame and the Price Column, 
        And creates the new column as Price Index for annual entries.'''
    
    df['Price Index'] = np.NaN
    df['Price Growth Perc'] = np.NaN
    
    for i in range(len(df)): 
        if (df.iloc[i]['Time'].month == 1) & (df.iloc[i]['Time'].year<2022):
            df['Price Index'].iloc[i] = 100
            df['Price Growth Perc'].iloc[i] = 0
            index_p = df.iloc[i][price_col]
        else: 
            df['Price Index'].iloc[i] = (100 * df.iloc[i][price_col])/index_p
            df['Price Growth Perc'].iloc[i] = (df.iloc[i]['Price Index'] - 100)/100

In [12]:
# apply price index and growth rate function on all dfs:

for df_ in df_list: 
    price_index_growth_rate(df_, price_col = 'Actual')

In [13]:
def index_avg_grp (df):
    
    '''This function calculate the average of 'Price Index' and 'Price Growth Percentage' for each
    calendar month over entire time preiod. i.e. average of March price growth perc.
    '''
    
    df['Price Index Avg'] = np.NaN
    df['Price Growth Avg'] = np.NaN
    
    ind_avg = df.groupby(['Month Name'])['Price Index'].mean().reset_index()
    grw_avg = df.groupby(['Month Name'])['Price Growth Perc'].mean().reset_index()
    
    df['Price Index Avg'] = df['Month Name'].map(ind_avg.set_index('Month Name')['Price Index']).fillna('')
    df['Price Growth Avg'] = df['Month Name'].map(grw_avg.set_index('Month Name')['Price Growth Perc']).fillna('')
    
    for i in range(len(df)):
        
        if np.isnan(df['Price Index'].iloc[i]):
            df['Price Index Avg'].iloc[i] = np.NaN
            df['Price Growth Avg'].iloc[i] = np.NaN

In [14]:
# apply price index and growth rate grouper function on all dfs:

for df_ in df_list: 
    index_avg_grp(df_)

In [15]:
df_alm.head(3)

,Time,Actual,Prediction ARIMA,lower_bond,higher_bond,Date,County,Year,Month,Quarter,Month Name,Price Index,Price Growth Perc,Price Index Avg,Price Growth Avg
0,2009-01-01,324170.0,NaN,NaN,NaN,Jan 01 2009,Alameda,2009,1,1,January,100.000000,0.000000,100.000000,0.000000
1,2009-02-01,327370.0,NaN,NaN,NaN,Feb 01 2009,Alameda,2009,2,1,February,100.987136,0.009871,103.772064,0.037721
2,2009-03-01,316440.0,NaN,NaN,NaN,Mar 01 2009,Alameda,2009,3,1,March,97.615449,-0.023846,110.700040,0.107000


### 6 -  Month to Month Growth Rate and Annual Growth Rate

In [16]:
# A - Create Month to Month Growth Rate for each year

def mnth_mnth_grw (df, price_col = 'Actual'):
    
    df['Month to Month Growth'] = np.NaN
    
    for i in range(len(df)):
            
        df['Month to Month Growth'].iloc[i] = (df[price_col].iloc[i]/df[price_col].iloc[i-1])-1
    
    # for the first month I used the Dec 2008 dataset
    county_name = df['County'].unique()[0]
    df['Month to Month Growth'].iloc[0] = (df[price_col].iloc[0]/Dec_2008_df[county_name].iloc[0])-1

In [17]:
# apply price index and growth rate grouper function on all dfs:

for df_ in df_list: 
    mnth_mnth_grw(df_, price_col ='Actual')

In [18]:
# B - Create Month to Month Growth Rate Average for each month (grouping average)

def avg_mnth_mnth_grp (df): 
    
    df['Month to Month Growth Avg'] = np.NaN
    
    mnth_grw_avg = df.groupby(['Month Name'])['Month to Month Growth'].mean().reset_index()
    
    df['Month to Month Growth Avg'] = df['Month Name'].map(mnth_grw_avg.set_index('Month Name')['Month to Month Growth']).fillna('')

    
    for i in range(len(df)):
        
        if np.isnan(df['Actual'].iloc[i]):
            df['Month to Month Growth Avg'].iloc[i] = np.NaN

In [19]:
# apply price index and growth rate grouper function on all dfs:

for df_ in df_list: 
    avg_mnth_mnth_grp(df_)

In [20]:
df_alm.head(4)

,Time,Actual,Prediction ARIMA,lower_bond,higher_bond,Date,County,Year,Month,Quarter,Month Name,Price Index,Price Growth Perc,Price Index Avg,Price Growth Avg,Month to Month Growth,Month to Month Growth Avg
0,2009-01-01,324170.0,NaN,NaN,NaN,Jan 01 2009,Alameda,2009,1,1,January,100.000000,0.000000,100.000000,0.000000,-0.119677,-0.046672
1,2009-02-01,327370.0,NaN,NaN,NaN,Feb 01 2009,Alameda,2009,2,1,February,100.987136,0.009871,103.772064,0.037721,0.009871,0.037721
2,2009-03-01,316440.0,NaN,NaN,NaN,Mar 01 2009,Alameda,2009,3,1,March,97.615449,-0.023846,110.700040,0.107000,-0.033387,0.067470
3,2009-04-01,337100.0,NaN,NaN,NaN,Apr 01 2009,Alameda,2009,4,2,April,103.988648,0.039886,114.820939,0.148209,0.065289,0.037353


### 7 - Quarterly Growth Rate and Average of Quarterly Growth

In [21]:
# A - Create Quarterly Growth Rate for each year

def qrtl_grw (df, price_col = 'Actual'):
    
    county_name = df['County'].unique()[0]
    d = Dec_2008_df[county_name].iloc[0]
    
    df['Quarter Growth'] = np.NaN
    
    for i in range(len(df)):
    
        if (i+1)%3 == 0 : 
            n = df[price_col].iloc[i]
            r = (n / d) - 1 
            df['Quarter Growth'].iloc[i] = r
            d = n

In [22]:
# apply quarterly growth and growth rate grouper function on all dfs:

for df_ in df_list: 
    qrtl_grw(df_, price_col ='Actual')

In [23]:
# B - Create Quarterly Growth Rate Average for each quarter (grouping average)

def avg_qrt_qrt_grp (df): 
    
    df['Quarterly Growth Avg'] = np.NaN
    
    qrt_grw_avg = df.groupby(['Quarter'])['Quarter Growth'].mean().reset_index()
    
    df['Quarterly Growth Avg'] = df['Quarter'].map(qrt_grw_avg.set_index('Quarter')['Quarter Growth']).fillna('')

    
    for i in range(len(df)):
        
        if np.isnan(df['Actual'].iloc[i]):
            df['Quarterly Growth Avg'].iloc[i] = np.NaN

In [24]:
# apply quarterly growth and growth rate grouper function on all dfs:

for df_ in df_list: 
    avg_qrt_qrt_grp(df_)

In [25]:
# concat all dfs 

df_all_counties = pd.concat([df_alm, df_con, df_mar, df_nap, df_sf, df_smt, df_scl, df_sol, df_son])

## Plot 2 

In [26]:
# Slider 
input_slider = alt.binding_range(name='Year    ', min=2009 , max=2021 , step=1)
select_year = alt.selection_single(fields=['Year'],
                                    bind=input_slider, init={'Year': 2017})

# Drop Down Column
input_dropdown = alt.binding_select(options=['Alameda','Contra Costa', 'Marin', 'Napa', 'San Francisco',
                                            'San Mateo','Santa Clara','Solano','Sonoma'], name="County    ")
select_county = alt.selection_single(fields=['County'], bind=input_dropdown, init={'County': "San Francisco"})



#### CHART 1 #####

base1 = alt.Chart(df_all_counties, title="12-Month Price Index").encode(
    x=alt.X('Month Name:O', title ='Month',  
            sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']),
     tooltip = [alt.Tooltip('Month Name:O',title='Month'),
               alt.Tooltip('Price Index:Q', title='Price Index',format='.2f'),
              alt.Tooltip('Price Index Avg:Q', title ='All Time Avg Price Index',format='.2f')])

bar1 = base1.mark_bar(stroke='#5276A7').encode(
    y=alt.Y('Price Index:Q', title='Price Index', scale=alt.Scale(domain=(72,150))),
    color=alt.condition(
        alt.datum['Price Index'] >= 100,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
)

line1 = base1.mark_line(color='red',opacity=0.20, point={
          "filled": True,
          "fill": "red",
          "opacity": 0.7}
).encode(
    y = alt.Y('Price Index Avg:Q', title='', )

)

ch1 = alt.layer(bar1 + line1).add_selection(
    select_year
).transform_filter(
    select_year
).add_selection(
    select_county
).transform_filter(
    select_county
).properties(
    height=270,
    width = 255)



###### CHART 2 ######



base2 = alt.Chart(df_all_counties, title="Monthly Price Growth (%)").encode(
    x=alt.X('Month Name:O', title ='Month',  
           sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']),
    tooltip = [alt.Tooltip('Month Name:O',title='Month'),
               alt.Tooltip('Month to Month Growth:Q', title='Growth Rate',format='.2%'),
              alt.Tooltip('Month to Month Growth Avg:Q', title ='All Time Avg Rate',format='.2%')]
)


bar2 = base2.mark_bar(stroke='#5276A7', ).encode(
    y= alt.Y('Month to Month Growth:Q', title='Growth Percentage', axis=alt.Axis(format='%')),
    color=alt.condition(
        alt.datum['Month to Month Growth'] > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
)   

line2 = base2.mark_line(color='red',opacity= 0.20, point={
          "filled": True,
          "fill": "red",
          "opacity": 0.7
    }).encode(
    y = alt.Y('Month to Month Growth Avg:Q', title='',scale=alt.Scale(domain=(-.32,0.32))),
)

ch2 = alt.layer(bar2 + line2).add_selection(
    select_year
).transform_filter(
    select_year
).add_selection(
    select_county
).transform_filter(
    select_county
).properties(
    height=270,
    width = 255)



#####  CHART 3 ######



base3 = alt.Chart(df_all_counties, title="Quarterly Price Growth (%)").transform_calculate(
    line3="'All Time Average'").encode(
    x=alt.X('Quarter:O', title ='Quarter', sort=None, axis=alt.Axis(titlePadding=28)),
    tooltip = [alt.Tooltip('Quarter:O',title='Quarter'),
               alt.Tooltip('Quarter Growth:Q', title='Growth Rate',format='.2%'),
            alt.Tooltip('Quarterly Growth Avg:Q', title ='All Time Avg Rate',format='.2%')]


)


scale3 = alt.Scale(domain=["All Time Average"], range=['red'])


bar3 = base3.mark_bar(stroke='#5276A7',size=30).encode(
    y= alt.Y('Quarter Growth:Q', title='Growth Percentage', axis=alt.Axis(format='%')),
    color=alt.condition(
        alt.datum['Quarter Growth'] > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
)



line3 = base3.mark_line(color='red', opacity=0.20, point={
        "filled" : True,
        "fill": "red",
        "opacity":0.7
    }).encode(
    y = alt.Y('Quarterly Growth Avg:Q', title='',scale=alt.Scale(domain=(-.32,0.32)),),
    color = alt.Color('line3:N',scale=scale3, title=''),
    
)



ch3 = alt.layer(bar3 + line3).add_selection(
    select_year
).transform_filter(
    select_year
).add_selection(
    select_county
).transform_filter(
    select_county
).properties(
    height=270,
    width = 255)


###  3 CHARTS + CONFIGURATION  ###

chart_b = (ch1 | ch2 | ch3).configure_title(fontSize=16,font='Helvetica Sans', color='dimgrey',  dy=-10
    ).configure_legend(labelFontSize=12, labelColor='k'
    ).configure_axisY(labelPadding=5, titleX=-50, labelFontSize=12, titleFontSize=14, titleColor = 'dimgray', titleFont = 'Helvetica Sans' , labelColor = 'grey'
    ).configure_axisX(grid=False, titleY=70, titleFontSize=14, labelFontSize=12, titleColor = 'dimgray', titleFont = 'Helvetica Sans', labelColor = 'grey'
    ).configure_axis(gridOpacity = 0.6
    ).configure_concat(spacing=20
)

In [27]:
chart_b

alt.HConcatChart(...)

In [28]:
chart_b.save('chart_b.html')